In [1]:
import pandas as pd

In [2]:
# Read in csv files
terminations = pd.read_csv("MembershipTerminations.csv")
terminations = terminations[terminations["TerminationReason"] == "membership_cancelled"]
usage_types = pd.read_csv("UsageTypes.csv")
visits_per_month = pd.read_csv("VisitsPerMonth.csv")

In [3]:
df = pd.merge(terminations, usage_types, on = "User_id", how = "inner")
df = pd.merge(df, visits_per_month, on = ["Id", "User_id"], how = "inner")
undisclosed_cols = ["Gender", "Ethnicity", "IsDisabled", "Persona", "output_area_classification_supergroup"]
df[undisclosed_cols] = df[undisclosed_cols].fillna("Not Given")
df["DiscountType"] = df["DiscountType"].fillna("No Discount")
df["output_area_classification_supergroup"] = df["output_area_classification_supergroup"].replace({"(pseudo) CI, IoM": "Not Given"})

# Gender mapping
gender_map = {
    'Male': 'Male',
    'M': 'Male',
    'Female': 'Female',
    'F': 'Female',
    'Non Binary': 'Non Binary',
    'Other': 'Non Binary',
    
    'Prefer Not To Say': 'Not Given',
    'Unknown': 'Not Given',
    'Not Given': 'Not Given'
}

# Apply mapping, default to 'not given' if value is not in the map
df['Gender'] = df['Gender'].map(gender_map).fillna('Not Given')

# Filter age to between 0 and 100
df = df[(df['MemberAge'] >= 0) & (df['MemberAge'] <= 100)]

# Filter VisitsPerMonth to between 0 and 50
df = df[(df['VisitsPerMonth'] >= 0) & (df['VisitsPerMonth'] <= 50)]

df = df.dropna()

In [4]:
for col in df.columns:
    print(f"{col}: {df[col].unique()}")

Id: [    512    3343    4456 ... 4208623 4220020 4294591]
User_id: [    910    3782    4913 ... 6565947 6420184 6438378]
MemberAge: [43. 33. 42. 30. 75. 76. 62. 52. 51. 67. 41. 84. 20. 57. 45. 55. 47. 71.
 63. 68. 39. 38. 28. 73. 31. 59. 61. 26. 70. 60. 22. 34. 23. 25. 18. 24.
 54. 50. 36. 48. 64. 37. 58. 65. 35. 27. 53. 90. 49. 46. 77. 44. 29. 56.
 32. 40. 78. 21. 79. 81. 74. 66. 72. 19. 69.  3. 80. 17. 16. 82. 14. 89.
 91. 11. 83. 13. 86. 12. 15. 87. 85.  6.  5.  7. 10. 94.  4.  8. 88. 92.
  9. 93.  2.  1. 95. 96. 98. 97.  0.]
Gender: ['Male' 'Female' 'Not Given' 'Non Binary']
Ethnicity: ['White' 'Not Given' 'Mixed or Multiple ethnic groups'
 'Black, Black British, Black Welsh, Caribbean or African'
 'Asian, Asian British or Asian Welsh' 'Other ethnic group']
IsDisabled: ['No' 'Not Given' 'Yes']
Membership: ['Better Health Centre' 'Better Swim' 'Better Health Partnership'
 'Better Health UK' 'Better Health Corporate' 'Better Student'
 'Better Health Centre Junior' 'Better Racquets' '

In [5]:
df.to_csv("Prepared Data.csv", index = False)